In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
!pip3 install neuralprophet[live]

     |████████████████████████████████| 55 kB 2.4 MB/s 
     |████████████████████████████████| 155 kB 10.2 MB/s 
  Attempting uninstall: holidays
    Found existing installation: holidays 0.10.5.2
    Uninstalling holidays-0.10.5.2:
      Successfully uninstalled holidays-0.10.5.2


In [3]:
from neuralprophet import NeuralProphet
from neuralprophet import set_random_seed

In [19]:
dsn=pd.read_csv(r"/content/Train Set.csv",index_col="Region_Name")

In [20]:
dsn

,Month 1,Month 2,Month 3,Month 4,Month 5,Month 6,Month 7,Month 8,Month 9,Month 10,Month 11,Month 12,Month 13,Month 14,Month 15,Month 16,Month 17,Month 18,Month 19,Month 20,Month 21,Month 22,Month 23,Month 24,Month 25,Month 26,Month 27,Month 28,Month 29,Month 30,Month 31,Month 32,Month 33,Month 34,Month 35,Month 36,Month 37,Month 38,Month 39,Month 40,Month 41,Month 42,Month 43,Month 44,Month 45,Month 46,Month 47,Month 48,Month 49,Month 50,Month 51,Month 52,Month 53,Month 54,Month 55,Month 56,Month 57,Month 58,Month 59,Month 60,Month 61,Month 62,Month 63,Month 64,Month 65,Month 66,Month 67,Month 68,Month 69,Month 70,Month 71,Month 72
Region_Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Region 1,0.154324,0.122901,0.127583,0.501557,0.103551,0.051983,0.140272,0.157586,0.182546,0.181801,-0.013496,0.213958,0.202585,0.256381,0.327656,0.277944,0.200623,0.141673,0.133457,0.092708,0.124225,0.199409,0.211584,0.256088,0.046845,0.050224,0.044738,0.049602,0.185713,0.141933,0.032244,0.101614,0.202953,0.185604,0.083222,0.097904,0.241120,0.264636,0.253286,0.422999,0.464442,0.316720,0.224314,0.338289,0.378352,0.520046,0.391280,0.314726,0.270200,0.362765,0.376728,0.394986,0.347692,0.351117,0.375120,0.289649,0.372379,0.398122,0.362610,0.263766,0.275622,0.252173,0.336913,0.321828,0.321895,0.337018,0.121581,0.234171,0.294693,0.373754,0.333642,0.242176
Region 2,0.119708,0.119831,0.112970,0.326049,0.098852,0.041502,0.116759,0.136964,0.178476,0.162621,-0.017723,0.227958,0.181563,0.247475,0.488062,0.290772,0.196146,0.139117,0.108014,0.091059,0.122529,0.202530,0.206454,0.243916,0.030216,0.057625,0.037567,0.047221,0.175260,0.121241,-0.009893,0.066981,0.191505,0.170095,0.057635,0.075968,0.216270,0.233841,0.245432,0.407445,0.437820,0.332397,0.238877,0.318503,0.366288,0.502219,0.360538,0.294045,0.241370,0.334915,0.359444,0.378980,0.306557,0.350775,0.342869,0.272034,0.344482,0.372061,0.318767,0.221102,0.221851,0.191496,0.283412,0.302669,0.306597,0.327412,0.124563,0.218367,0.275179,0.344997,0.300503,0.215147
Region 3,0.178236,0.165497,0.190754,0.321733,0.203126,0.048345,0.180064,0.176990,0.224173,0.223810,0.070306,0.314094,0.253627,0.317007,0.493548,0.356269,0.301312,0.295783,0.213944,0.225560,0.199628,0.295161,0.301142,0.369074,0.148873,0.148648,0.141286,0.176595,0.298081,0.247686,0.073358,0.167071,0.369198,0.273285,0.166522,0.232556,0.324115,0.337240,0.328820,0.488371,0.452943,0.299717,0.276886,0.353269,0.391376,0.575888,0.412834,0.404446,0.315161,0.413679,0.436428,0.452330,0.355533,0.397592,0.429922,0.315471,0.425693,0.437532,0.401876,0.320588,0.310557,0.285627,0.331988,0.377643,0.384990,0.362927,0.230516,0.263643,0.296751,0.400269,0.360406,0.335923
Region 4,0.225016,0.220602,0.224240,0.330916,0.268934,0.171322,0.243672,0.219831,0.242115,0.227575,0.068486,0.300100,0.260220,0.339176,0.408422,0.367631,0.374176,0.385319,0.200377,0.308989,0.264775,0.300382,0.328013,0.395487,0.204112,0.210327,0.230464,0.254440,0.428539,0.309254,0.155241,0.227263,0.471488,0.294128,0.162800,0.191990,0.303951,0.360914,0.397259,0.534679,0.493992,0.373266,0.296719,0.367477,0.400149,0.625336,0.415876,0.373174,0.305040,0.419634,0.408965,0.453475,0.325201,0.445034,0.414192,0.308857,0.381268,0.432136,0.366376,0.264211,0.271887,0.248869,0.315252,0.329762,0.352229,0.368514,0.192878,0.228902,0.291502,0.364874,0.311379,0.271102
Region 5,0.318976,0.316835,0.328486,0.483001,0.380898,0.292074,0.268395,0.326809,0.344086,0.278257,0.187874,0.588132,0.376303,0.429040,0.484136,0.547735,0.458580,0.588121,0.336758,0.398324,0.404609,0.351204,0.421711,0.620374,0.446481,0.354543,0.378399,0.423216,0.478113,0.386218,0.323202,0.331631,0.547656,0.383964,0.226795,0.392175,0.430364,0.439188,0.421460,0.672617,0.556797,0.527481,0.380318,0.429014,0.463007,0.671208,0.487795,0.500731,0.328601,0.450884,0.471953,0.506876,0.388457,0.417775,0.470368,0.363075,0.410614,0.482598,0.408525,0.381367,0.336923,0.288830,0.366314,0.392924,0.370159,0.422587,0.282631,0.289498,0.368923,0.416527,0.3725

In [22]:
count=0
for i,row in dsn.iterrows():
    std=np.std(row)
    mean=np.mean(row)
    for k in range(1,73):
        if(  ((row["Month "+str(k)]-mean)/std)>4 ):
            row["Month "+str(k)]=mean+(std*4)
            count+=1
print(count)

185


In [23]:
dsnt=dsn.T

In [24]:
dsnt

Region_Name,Region 1,Region 2,Region 3,Region 4,Region 5,Region 6,Region 7,Region 8,Region 9,Region 10,Region 11,Region 12,Region 13,Region 14,Region 15,Region 16,Region 17,Region 18,Region 19,Region 20,Region 21,Region 22,Region 23,Region 24,Region 25,Region 26,Region 27,Region 28,Region 29,Region 30,Region 31,Region 32,Region 33,Region 34,Region 35,Region 36,Region 37,Region 38,Region 39,Region 40,...,Region 3876,Region 3877,Region 3878,Region 3879,Region 3880,Region 3881,Region 3882,Region 3883,Region 3884,Region 3885,Region 3886,Region 3887,Region 3888,Region 3889,Region 3890,Region 3891,Region 3892,Region 3893,Region 3894,Region 3895,Region 3896,Region 3897,Region 3898,Region 3899,Region 3900,Region 3901,Region 3902,Region 3903,Region 3904,Region 3905,Region 3906,Region 3907,Region 3908,Region 3909,Region 3910,Region 3911,Region 3912,Region 3913,Region 3914,Region 3915
Month 1,0.154324,0.119708,0.178236,0.225016,0.318976,0.049922,0.219802,0.069055,0.186616,0.053525,0.190577,0.158609,0.165217,0.069157,0.157302,0.072315,0.024757,0.180025,0.206183,0.050629,0.212532,0.137973,0.075377,0.172759,3.587049,0.315296,0.873200,0.157227,0.282547,0.164005,0.083445,0.012357,0.245894,0.065887,0.154048,0.342258,0.312587,0.105453,0.100788,0.086042,...,0.444992,0.276426,0.495871,0.754557,0.317987,2.447478,1.552072,1.120383,0.908094,0.878383,1.043914,1.035110,16.265325,0.430467,0.492921,0.281649,0.599147,1.500303,0.363861,0.315941,0.344324,1.010537,0.387550,0.196002,2.231286,0.353811,0.785468,0.290261,0.468493,0.408514,0.154372,0.338739,0.253205,0.233119,0.199864,3.400484,0.374751,0.463527,1.641510,0.307089
Month 2,0.122901,0.119831,0.165497,0.220602,0.316835,0.082337,0.175329,0.059602,0.185539,0.050294,0.147166,0.141044,0.159698,0.046176,0.159635,0.071109,0.050729,0.172872,0.182795,0.067780,0.163495,0.135268,0.082645,0.152321,3.001145,0.291904,0.717435,0.128892,0.217566,0.224851,0.074453,0.041007,0.201204,0.089552,0.128339,0.267466,0.272249,0.100428,0.142518,0.097482,...,0.709842,0.441313,0.690150,1.022267,0.457990,3.477949,1.719477,1.325422,0.812730,1.168977,1.671550,1.295669,20.465496,0.657171,0.559615,0.375383,0.623699,1.815890,0.472307,0.594319,0.634195,1.128955,0.568140,0.292100,2.527318,0.489000,0.963027,0.449931,0.597720,0.489851,0.180818,0.424401,0.336452,0.290894,0.328968,3.999567,0.478654,0.571315,1.649937,0.451604
Month 3,0.127583,0.112970,0.190754,0.224240,0.328486,0.083182,0.178562,0.071814,0.207523,0.060354,0.131250,0.162525,0.175699,0.056705,0.511734,0.090473,0.054843,0.195426,0.231616,0.086471,0.221174,0.185897,0.163404,0.220260,3.025418,0.327416,0.704960,0.444618,0.294857,0.219886,0.442780,0.076713,0.237413,0.128469,0.214407,0.344732,0.524357,0.322261,0.158039,0.229757,...,0.534610,0.402494,0.662206,1.033323,0.441139,3.416021,1.853863,1.345858,1.059389,1.113182,1.193873,1.326013,20.536352,0.684776,0.668832,0.328184,0.618134,1.717311,0.527924,0.539337,0.531636,1.247042,0.516666,0.282652,2.694897,0.460567,0.957489,0.428220,0.617278,0.519781,0.407456,0.485308,0.369691,0.301755,0.271360,3.308452,0.501871,0.644025,2.110449,0.448063
Month 4,0.501557,0.326049,0.321733,0.330916,0.483001,0.156701,0.245769,0.123014,0.364449,0.115983,0.198793,0.216700,0.238456,0.145198,0.665948,0.155938,0.155571,0.271052,0.285454,0.188782,0.539270,0.180723,0.174683,0.282689,2.345321,0.605174,0.600299,0.375266,0.313095,0.235294,0.472638,0.161510,0.261674,0.254538,0.200919,0.377825,0.485539,0.217263,0.191939,0.216380,...,0.489195,0.407036,0.673595,0.954770,0.451388,2.840427,1.508850,1.090429,0.934530,1.093475,1.022606,1.070847,15.696556,0.592267,0.601639,0.399364,0.589935,1.579690,0.497742,0.531691,0.532295,1.126943,0.521146,0.364811,2.432936,0.484782,0.965312,0.413396,0.607717,0.568379,0.403261,0.473134,0.376600,0.358181,0.447163,2.636698,0.481275,0.636814,1.642345,0.464692
Month 5,0.103551,0.098852,0.203126,0.268934,0.380898,0.013289,0.139204,0.078820,0.165327,0.072806,0.116737,0.122395,0.143661,0.042786,0.136179,0.056539,0.031122,0.109780,0.120330,0.

In [25]:
dsnt.index=range(1,73)
dts=pd.Series(["2013-1-1"])
dts=pd.to_datetime(dts)

In [26]:
from dateutil.relativedelta import relativedelta
dts.iloc[0]+relativedelta(months=1)

Timestamp('2013-02-01 00:00:00')

In [27]:
for i in range(1,72):
    dts[i]=dts[i-1]+relativedelta(months=1)

In [28]:
dsnt.index=dts
dsnt

Region_Name,Region 1,Region 2,Region 3,Region 4,Region 5,Region 6,Region 7,Region 8,Region 9,Region 10,Region 11,Region 12,Region 13,Region 14,Region 15,Region 16,Region 17,Region 18,Region 19,Region 20,Region 21,Region 22,Region 23,Region 24,Region 25,Region 26,Region 27,Region 28,Region 29,Region 30,Region 31,Region 32,Region 33,Region 34,Region 35,Region 36,Region 37,Region 38,Region 39,Region 40,...,Region 3876,Region 3877,Region 3878,Region 3879,Region 3880,Region 3881,Region 3882,Region 3883,Region 3884,Region 3885,Region 3886,Region 3887,Region 3888,Region 3889,Region 3890,Region 3891,Region 3892,Region 3893,Region 3894,Region 3895,Region 3896,Region 3897,Region 3898,Region 3899,Region 3900,Region 3901,Region 3902,Region 3903,Region 3904,Region 3905,Region 3906,Region 3907,Region 3908,Region 3909,Region 3910,Region 3911,Region 3912,Region 3913,Region 3914,Region 3915
2013-01-01,0.154324,0.119708,0.178236,0.225016,0.318976,0.049922,0.219802,0.069055,0.186616,0.053525,0.190577,0.158609,0.165217,0.069157,0.157302,0.072315,0.024757,0.180025,0.206183,0.050629,0.212532,0.137973,0.075377,0.172759,3.587049,0.315296,0.873200,0.157227,0.282547,0.164005,0.083445,0.012357,0.245894,0.065887,0.154048,0.342258,0.312587,0.105453,0.100788,0.086042,...,0.444992,0.276426,0.495871,0.754557,0.317987,2.447478,1.552072,1.120383,0.908094,0.878383,1.043914,1.035110,16.265325,0.430467,0.492921,0.281649,0.599147,1.500303,0.363861,0.315941,0.344324,1.010537,0.387550,0.196002,2.231286,0.353811,0.785468,0.290261,0.468493,0.408514,0.154372,0.338739,0.253205,0.233119,0.199864,3.400484,0.374751,0.463527,1.641510,0.307089
2013-02-01,0.122901,0.119831,0.165497,0.220602,0.316835,0.082337,0.175329,0.059602,0.185539,0.050294,0.147166,0.141044,0.159698,0.046176,0.159635,0.071109,0.050729,0.172872,0.182795,0.067780,0.163495,0.135268,0.082645,0.152321,3.001145,0.291904,0.717435,0.128892,0.217566,0.224851,0.074453,0.041007,0.201204,0.089552,0.128339,0.267466,0.272249,0.100428,0.142518,0.097482,...,0.709842,0.441313,0.690150,1.022267,0.457990,3.477949,1.719477,1.325422,0.812730,1.168977,1.671550,1.295669,20.465496,0.657171,0.559615,0.375383,0.623699,1.815890,0.472307,0.594319,0.634195,1.128955,0.568140,0.292100,2.527318,0.489000,0.963027,0.449931,0.597720,0.489851,0.180818,0.424401,0.336452,0.290894,0.328968,3.999567,0.478654,0.571315,1.649937,0.451604
2013-03-01,0.127583,0.112970,0.190754,0.224240,0.328486,0.083182,0.178562,0.071814,0.207523,0.060354,0.131250,0.162525,0.175699,0.056705,0.511734,0.090473,0.054843,0.195426,0.231616,0.086471,0.221174,0.185897,0.163404,0.220260,3.025418,0.327416,0.704960,0.444618,0.294857,0.219886,0.442780,0.076713,0.237413,0.128469,0.214407,0.344732,0.524357,0.322261,0.158039,0.229757,...,0.534610,0.402494,0.662206,1.033323,0.441139,3.416021,1.853863,1.345858,1.059389,1.113182,1.193873,1.326013,20.536352,0.684776,0.668832,0.328184,0.618134,1.717311,0.527924,0.539337,0.531636,1.247042,0.516666,0.282652,2.694897,0.460567,0.957489,0.428220,0.617278,0.519781,0.407456,0.485308,0.369691,0.301755,0.271360,3.308452,0.501871,0.644025,2.110449,0.448063
2013-04-01,0.501557,0.326049,0.321733,0.330916,0.483001,0.156701,0.245769,0.123014,0.364449,0.115983,0.198793,0.216700,0.238456,0.145198,0.665948,0.155938,0.155571,0.271052,0.285454,0.188782,0.539270,0.180723,0.174683,0.282689,2.345321,0.605174,0.600299,0.375266,0.313095,0.235294,0.472638,0.161510,0.261674,0.254538,0.200919,0.377825,0.485539,0.217263,0.191939,0.216380,...,0.489195,0.407036,0.673595,0.954770,0.451388,2.840427,1.508850,1.090429,0.934530,1.093475,1.022606,1.070847,15.696556,0.592267,0.601639,0.399364,0.589935,1.579690,0.497742,0.531691,0.532295,1.126943,0.521146,0.364811,2.432936,0.484782,0.965312,0.413396,0.607717,0.568379,0.403261,0.473134,0.376600,0.358181,0.447163,2.636698,0.481275,0.636814,1.642345,0.464692
2013-05-01,0.103551,0.098852,0.203126,0.268934,0.380898,0.013289,0.139204,0.078820,0.165327,0.072806,0.116737,0.122395,0.143661,0.042786,0.136179,0.056539,0.031122,0.109

In [65]:
runner=pd.DataFrame({"ds":dsnt.index,"y":dsnt["Region 1"].values})

In [66]:
model = NeuralProphet(daily_seasonality=True,weekly_seasonality=True)
model.add_country_holidays(country_name="IN")
metrics = model.fit(runner, freq="D")

INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 347


  0%|          | 0/195 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 2.98E-02, min: 1.01E+00
INFO - (NP.utils_torch.lr_range_test) - learning rate range test selected lr: 3.12E-01
Epoch[347/347]: 100%|██████████| 347/347 [00:05<00:00, 58.74it/s, SmoothL1Loss=0.00906, MAE=0.0459, MSE=0.00345, RegLoss=0]


In [67]:
future = model.make_future_dataframe(runner, periods=16, n_historic_predictions=True)
forecast = model.predict(future)


In [68]:
forecast["yhat1"].tail(15)

73    0.353063
74    0.323966
75    0.344958
76    0.389690
77    0.397458
78    0.489572
79    0.443054
80    0.551903
81    0.509337
82    0.514864
83    0.542299
84    0.531120
85    0.602839
86    0.534695
87    0.620913
Name: yhat1, dtype: float64

In [49]:
ansdf=pd.DataFrame(index=["Month "+str(i) for i in range(73,88)])
ansdf

""
Month 73
Month 74
Month 75
Month 76
Month 77
Month 78
Month 79
Month 80
Month 81
Month 82


In [ ]:
for i in range(1,3916):
  runner=pd.DataFrame({"ds":dsnt.index,"y":dsnt["Region "+str(i)].values})
  model = NeuralProphet(daily_seasonality=True,weekly_seasonality=True)
  model.add_country_holidays(country_name="IN")
  metrics = model.fit(runner, freq="D")
  future = model.make_future_dataframe(runner, periods=16, n_historic_predictions=True)
  forecast = model.predict(future)
  ansdf["Region "+str(i)]=forecast["yhat1"].tail(15).values
  print(i)




INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 347


  0%|          | 0/195 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 4.57E-02, min: 1.92E+00
INFO - (NP.utils_torch.lr_range_test) - learning rate range test selected lr: 5.52E-01
Epoch[347/347]: 100%|██████████| 347/347 [00:05<00:00, 59.29it/s, SmoothL1Loss=0.00908, MAE=0.0462, MSE=0.00346, RegLoss=0]
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 347


  0%|          | 0/195 [00:00<?, ?it/s]

1


...

145


INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 3.32E-02, min: 7.34E-01
INFO - (NP.utils_torch.lr_range_test) - learning rate range test selected lr: 2.62E-01
Epoch[347/347]: 100%|██████████| 347/347 [00:07<00:00, 46.44it/s, SmoothL1Loss=0.00614, MAE=0.0504, MSE=0.00422, RegLoss=0]
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 347


  0%|          | 0/195 [00:00<?, ?it/s]

146


INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 8.67E-02, min: 5.33E-01
INFO - (NP.utils_torch.lr_range_test) - learning rate range test selected lr: 2.91E-01
Epoch[215/347]:  61%|██████    | 211/347 [00:04<00:02, 48.19it/s, SmoothL1Loss=0.0116, MAE=0.062, MSE=0.00599, RegLoss=0]  

In [70]:
ansdf

,Region 1,Region 2,Region 3,Region 4,Region 5
Month 73,0.347436,0.310383,0.388272,0.337859,0.530948
Month 74,0.318311,0.302549,0.382694,0.369122,0.570816
Month 75,0.337058,0.317044,0.389767,0.381175,0.623725
Month 76,0.381678,0.349839,0.412741,0.427068,0.668502
Month 77,0.383223,0.371932,0.441100,0.469865,0.716788
Month 78,0.473609,0.456241,0.503278,0.545703,0.798578
Month 79,0.428657,0.405228,0.458135,0.512374,0.802593
Month 80,0.535799,0.485498,0.522125,0.574792,0.855444
Month 81,0.493954,0.465752,0.506409,0.589740,0.870590
Month 82,0.498087,0.466569,0.501996,0.583138,0.895586


In [72]:
askd=pd.DataFrame({"Month":["Month "+str(i) for i in range(73,88)]})

In [73]:
for i in range(1,6):
  t="Region "+str(i)
  askd[t]=ansdf[t].values

In [74]:
askd

,Month,Region 1,Region 2,Region 3,Region 4,Region 5
0,Month 73,0.347436,0.310383,0.388272,0.337859,0.530948
1,Month 74,0.318311,0.302549,0.382694,0.369122,0.570816
2,Month 75,0.337058,0.317044,0.389767,0.381175,0.623725
3,Month 76,0.381678,0.349839,0.412741,0.427068,0.668502
4,Month 77,0.383223,0.371932,0.441100,0.469865,0.716788
5,Month 78,0.473609,0.456241,0.503278,0.545703,0.798578
6,Month 79,0.428657,0.405228,0.458135,0.512374,0.802593
7,Month 80,0.535799,0.485498,0.522125,0.574792,0.855444
8,Month 81,0.493954,0.465752,0.506409,0.589740,0.870590
9,Month 82,0.498087,0.466569,0.501996,0.583138,0.895586


In [ ]:
askd.to_csv(r"/content/ans5.csv")